NegaMax Implementation for MiniMax

In [ ]:
def _value(self, board: chess.Board, depth: int) -> int:
    if score := self._evaluate(board):
        return score
    if depth == 0:
        return self._absolute_heuristic(board)

    scores = []
    for move in board.legal_moves:
        board.push(move)
        scores.append(-1 * self._value(board, depth - 1))
        board.pop()

    return max(scores)


MiniMaxEngine._value = _value


def _evaluate_move(self, board: chess.Board, move: chess.Move, depth: int):
    board.push(move)
    score = self._value(board, depth)
    score *= self.PLAYER_MULTIPLIER[board.turn]
    board.pop()
    return ScoredMove(score=score, move=move)


MiniMaxEngine._evaluate_move = _evaluate_move

A more efficient Zobrist Hasher. For even more performance incremental hashing needs to be implemented

In [ ]:
import chess.polyglot


class ZobristHasher(chess.polyglot.ZobristHasher):

    def hash_castling(self, board: chess.Board) -> int:
        zobrist_hash = 0

        if board.castling_rights & chess.BB_H1:
            zobrist_hash ^= self.array[768]
        if board.castling_rights & chess.BB_A1:
            zobrist_hash ^= self.array[768 + 1]
        if board.castling_rights & chess.BB_H8:
            zobrist_hash ^= self.array[768 + 2]
        if board.castling_rights & chess.BB_A8:
            zobrist_hash ^= self.array[768 + 3]

        return zobrist_hash

    def hash_board(self, board: chess.BaseBoard) -> int:
        zobrist_hash = 0

        for square in chess.scan_reversed(board.occupied_co[chess.BLACK]):
            zobrist_hash ^= self.array[128 * board.piece_type_at(square) - 128 +
                                       square]

        for square in chess.scan_reversed(board.occupied_co[chess.WHITE]):
            zobrist_hash ^= self.array[128 * board.piece_type_at(square) - 64 +
                                       square]

        return zobrist_hash

    def set_piece(self, hash):
        hash ^= self.array[64 * 3 + 8 * 10]  # Sample zrobist xor
        hash ^= self.array[64 * 3 + 8 * 10]  # Sample zrobist xor
        hash ^= self.array[64 * 3 + 8 * 10]  # Sample zrobist xor
        hash ^= self.array[64 * 3 + 8 * 10]  # Sample zrobist xor


zobrist_hash_old = chess.polyglot.ZobristHasher(
    chess.polyglot.POLYGLOT_RANDOM_ARRAY
)
zobrist_hash_new = ZobristHasher(chess.polyglot.POLYGLOT_RANDOM_ARRAY)

board = chess.Board()

# move = list(board.legal_moves)[0]
# board.push(move)

%timeit zobrist_hash_old.hash_board(board)
%timeit zobrist_hash_new.hash_board(board)

hash = zobrist_hash_new.hash_board(board)

%timeit zobrist_hash_new.set_piece(hash)

In [ ]:
# Performance of board with at least one move does make a difference (as push() is precalculating some things)
move = list(board.legal_moves)[0]
board.push(move)

%timeit board._transposition_key()
%timeit board._transposition_key()
%timeit board._transposition_key()

board.pop()

Performance single move

In [ ]:
class Test(IterativeAlphaBetaCached):

    def __init__(self):
        self._value_counter = 0
        self._value2_counter = 0
        self._evaluate_counter = 0
        self._absolute_heuristic_counter = 0
        super().__init__()

    def _value(self, *args, **kwarg):
        self._value_counter += 1
        return super()._value(*args, **kwarg)

    def _value2(self, *args, **kwarg):
        self._value2_counter += 1
        return super()._value2(*args, **kwarg)

    def _evaluate(self, *args, **kwarg):
        self._evaluate_counter += 1
        return super()._evaluate(*args, **kwarg)

    def _absolute_heuristic(self, *args, **kwargs):
        self._absolute_heuristic_counter += 1
        return super()._absolute_heuristic(*args, **kwargs)

In [ ]:
class Board(chess.Board):

    def __init__(self, *args, **kwarg):
        self.push_counter = 0
        super().__init__(*args, **kwarg)

    def push(self, *args, **kwarg):
        self.push_counter += 1
        super().push(*args, **kwarg)

In [ ]:
engine = Test()
board = Board("rnbqkbnr/p2ppppp/8/1p6/3PP3/8/PP3PPP/RNBQKBNR b Kkq - 0 1")
%time result = engine._evaluate_move(board, ev2[0].move, 5)
print(engine._value_counter)
print(engine._value2_counter)
print(engine._evaluate_counter)
print(engine._absolute_heuristic_counter)
print(board.push_counter)